https://thelaziestprogrammer.com/sharrington/math-of-machine-learning/solving-logreg-newtons-method

https://rstudio-pubs-static.s3.amazonaws.com/160015_b192ca9855e84b57814e785ebd034a5e.html

https://www.dotnetlovers.com/Article/225/logistic-regression-explained

https://medium.com/deep-math-machine-learning-ai/chapter-2-0-logistic-regression-with-math-e9cbb3ec6077

https://www.kaggle.com/code/elyas19/implement-logistic-regression-from-scratch/notebook

https://sonsnotation.blogspot.com/2020/11/2-logistic-regression.html

https://lee-jaejoon.github.io/stat-logistic/

In [1]:
import numpy as np
import pandas as pd
import random

# Data Generating

In [44]:
sample_size = 100
x = np.random.normal(0,1,sample_size)
pi = 0.3 + (1 * x)
p = 1 / (1 + np.exp(-pi))
y = np.random.binomial(1,p,sample_size) # 각각 매칭이 되는지
df = pd.DataFrame(list(zip(np.ones(100,int),x, y)),columns=['intercept','x','y'])

In [45]:
df

,intercept,x,y
0,1,0.337672,1
1,1,-0.239283,1
2,1,1.153391,1
3,1,1.404200,1
4,1,0.265838,1
...,...,...,...
95,1,-0.373829,0
96,1,-0.978608,0
97,1,0.805649,1
98,1,0.994090,1


# Newton's Method

In [46]:
# x = np.concatenate((df['x'].values.reshape(-1,1),df['intercept'].values.reshape(-1,1)), axis=1)
x = df[['intercept','x']].values
y = df['y'].values.reshape(-1,1)
weight_vec = np.random.normal(0,1,[2,1])
weight = [weight_vec]
while True:
    pi = 1/(1+np.exp(-x@weight_vec))
    grad = (1/len(df))*(x.T@(pi-y))
    
    # np.dot, @ 매우 느림 .dot이 빠름
    H = (x.T.dot(np.diag(pi.reshape(len(df)))).dot(np.diag((1-pi).reshape(len(df)))).dot(x))
    weight_vec = weight_vec - (np.linalg.inv(1/len(df)*H).T@grad) # np.linalg.pinv
    weight.append(weight_vec)
    print(np.round(weight_vec,2))
    
    if all(np.round(weight[-2],10) == np.round(weight_vec,10)): # 이전 가중치와 소수점 10자리까지 같으면 중지
        XWX = np.linalg.inv(H)
        break

[[0.65]
 [0.67]]
[[0.31]
 [1.14]]
[[0.3 ]
 [1.21]]
[[0.3 ]
 [1.21]]
[[0.3 ]
 [1.21]]
[[0.3 ]
 [1.21]]


# Predict

In [47]:
df['p'] = 1/(1+np.exp(-(weight_vec[1]*df['x'] + weight_vec[0])))
df['p'].fillna(0,inplace=True)

# cutoff
df.loc[df['p'] >= 0.5, 'result'] = 1
df.loc[df['p'] < 0.5, 'result'] = 0

In [48]:
df

,intercept,x,y,p,result
0,1,0.337672,1,0.670569,1.0
1,1,-0.239283,1,0.502468,1.0
2,1,1.153391,1,0.845757,1.0
3,1,1.404200,1,0.881466,1.0
4,1,0.265838,1,0.651013,1.0
...,...,...,...,...,...
95,1,-0.373829,0,0.461681,0.0
96,1,-0.978608,0,0.291467,0.0
97,1,0.805649,1,0.782324,1.0
98,1,0.994090,1,0.818792,1.0


# Library

In [49]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [50]:
model = LogisticRegression()
model.fit(df['x'].values.reshape(-1,1), df["y"])

LogisticRegression()

# Comparison

In [51]:
# Coefficient
print("Implementation:",weight[-1].T)
print("sklearn library:",model.intercept_,model.coef_)

Implementation: [[0.30055361 1.21481014]]
sklearn library: [0.29452055] [[1.12539326]]


In [52]:
# Score
pred = model.predict(df['x'].values.reshape(-1,1))
print("Implementation:",sum(df.result.values == df.y.values)/1)
print("sklearn library:",np.round(accuracy_score(df['y'],pred)*100,2))

Implementation: 69.0
sklearn library: 68.0


# Variance and Standard Error of $ \hat{\beta} $

In [53]:
np.diag(np.sqrt(XWX))

array([0.22985132, 0.28693678])

In [54]:
import statsmodels.api as sm
logit_model = sm.Logit(y, df[['intercept','x']]).fit()
logit_model.summary()

Optimization terminated successfully.
         Current function value: 0.563119
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                  100
Model:                          Logit   Df Residuals:                       98
Method:                           MLE   Df Model:                            1
Date:                Wed, 29 Jun 2022   Pseudo R-squ.:                  0.1759
Time:                        10:20:55   Log-Likelihood:                -56.312
converged:                       True   LL-Null:                       -68.331
Covariance Type:            nonrobust   LLR p-value:                 9.439e-07
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.3006      0.230      1.308      0.191      -0.150       0.751
x              1.2148      0.287      4.234      0.000       0.652       1.777
==============================================================================
"""

# Application of N-dimensional

In [55]:
sample_size = 100
x1 = np.random.normal(0,1,sample_size)
x2 = np.random.normal(0,1,sample_size)
pi = 0.3 + ((1 * x1)+(0.4 * x2))
p = 1 / (1 + np.exp(-pi))
y = np.random.binomial(1,p,sample_size)
df = pd.DataFrame(list(zip(np.ones(100,int),x1, x2, y)),columns=['intercept','x1','x2','y'])

In [56]:
# x = np.concatenate((df['x'].values.reshape(-1,1),df['intercept'].values.reshape(-1,1)), axis=1)
x = df[['intercept','x1','x2',]].values
y = df['y'].values.reshape(-1,1)
weight_vec = np.random.normal(0,1,[3,1])
weight = [weight_vec]

while True:
    pi = 1/(1+np.exp(-x@weight_vec))
    grad = (1/len(df))*(x.T@(pi-y))
    
    # np.dot, @ 매우 느림 .dot이 빠름
    H = x.T.dot(np.diag(pi.reshape(len(df)))).dot(np.diag((1-pi).reshape(len(df)))).dot(x)
    weight_vec = weight_vec - (np.linalg.inv((1/len(df))*H)).T@grad # np.linalg.pinv
    weight.append(weight_vec)
    print(np.round(weight_vec,2))
    
    if all(np.round(weight[-2],10) == np.round(weight_vec,10)): # 이전 가중치와 소수점 10자리까지 같으면 중지
        break

[[1.08]
 [1.42]
 [1.52]]
[[-0.78]
 [-0.11]
 [-1.04]]
[[0.78]
 [1.09]
 [1.21]]
[[-0.2 ]
 [ 0.52]
 [-0.27]]
[[0.16]
 [0.8 ]
 [0.3 ]]
[[0.16]
 [0.88]
 [0.29]]
[[0.17]
 [0.89]
 [0.29]]
[[0.17]
 [0.89]
 [0.29]]
[[0.17]
 [0.89]
 [0.29]]


In [57]:
model = LogisticRegression()
model.fit(df[['x1','x2',]].values.reshape(-1,2), df["y"])
print("sklearn library:", model.intercept_, model.coef_)
print("Implementation:",weight[-1].T)

sklearn library: [0.16006011] [[0.8309457  0.26811051]]
Implementation: [[0.16542088 0.88508007 0.29194162]]


In [58]:
import pandas as pd
from sklearn import datasets

# 유방암 데이터셋 로드
data = datasets.load_breast_cancer()
ex = pd.DataFrame(data.data, columns = data.feature_names)
ex['target'] = data.target
ex['intercept'] = 1
ex = ex[['intercept','mean radius', 'mean texture', 'mean area', 'mean symmetry','target']]
ex

,intercept,mean radius,mean texture,mean area,mean symmetry,target
0,1,17.99,10.38,1001.0,0.2419,0
1,1,20.57,17.77,1326.0,0.1812,0
2,1,19.69,21.25,1203.0,0.2069,0
3,1,11.42,20.38,386.1,0.2597,0
4,1,20.29,14.34,1297.0,0.1809,0
...,...,...,...,...,...,...
564,1,21.56,22.39,1479.0,0.1726,0
565,1,20.13,28.25,1261.0,0.1752,0
566,1,16.60,28.08,858.1,0.1590,0
567,1,20.60,29.33,1265.0,0.2397,0


In [59]:
from sklearn.preprocessing import StandardScaler
data = datasets.load_breast_cancer()
scaler = StandardScaler()
st = scaler.fit_transform(data.data)
ex = pd.DataFrame(st, columns = data.feature_names)
ex['target'] = data.target
ex['intercept'] = 1
ex = ex[['intercept','mean radius', 'mean texture', 'mean area', 'mean symmetry','target']]
ex

,intercept,mean radius,mean texture,mean area,mean symmetry,target
0,1,1.097064,-2.073335,0.984375,2.217515,0
1,1,1.829821,-0.353632,1.908708,0.001392,0
2,1,1.579888,0.456187,1.558884,0.939685,0
3,1,-0.768909,0.253732,-0.764464,2.867383,0
4,1,1.750297,-1.151816,1.826229,-0.009560,0
...,...,...,...,...,...,...
564,1,2.110995,0.721473,2.343856,-0.312589,0
565,1,1.704854,2.085134,1.723842,-0.217664,0
566,1,0.702284,2.045574,0.577953,-0.809117,0
567,1,1.838341,2.336457,1.735218,2.137194,0


In [60]:
ex.groupby('target').size()

target
0    212
1    357
dtype: int64

In [61]:
x = ex[['intercept','mean radius', 'mean texture', 'mean area', 'mean symmetry']].values
y = ex['target'].values.reshape(-1,1)
weight_vec = np.random.normal(0,1,[5,1])
weight = [weight_vec]
while True:
    pi = 1/(1+np.exp(-x@weight_vec))
    grad = (1/len(ex))*(x.T@(pi-y))
    
    # np.dot, @ 매우 느림 .dot이 빠름
    H = (x.T.dot(np.diag(pi.reshape(len(ex)))).dot(np.diag((1-pi).reshape(len(ex)))).dot(x))
    weight_vec = weight_vec - (np.linalg.inv(1/len(ex)*H).T@grad) # np.linalg.pinv
    weight.append(weight_vec)
    print(np.round(weight_vec,2))
    
    if all(np.round(weight[-2],10) == np.round(weight_vec,10)): # 이전 가중치와 소수점 10자리까지 같으면 중지
        break
        

[[ 0.14]
 [-3.65]
 [-0.9 ]
 [ 2.31]
 [-0.8 ]]
[[ 0.68]
 [-4.23]
 [-0.84]
 [ 1.87]
 [-1.01]]
[[ 0.77]
 [-3.96]
 [-1.02]
 [ 0.58]
 [-1.28]]
[[ 0.51]
 [-1.38]
 [-1.13]
 [-3.  ]
 [-1.44]]
[[ 0.2 ]
 [ 1.23]
 [-1.17]
 [-6.33]
 [-1.51]]
[[ 0.13]
 [ 1.76]
 [-1.18]
 [-7.03]
 [-1.52]]
[[ 0.13]
 [ 1.78]
 [-1.18]
 [-7.06]
 [-1.52]]
[[ 0.13]
 [ 1.78]
 [-1.18]
 [-7.06]
 [-1.52]]
[[ 0.13]
 [ 1.78]
 [-1.18]
 [-7.06]
 [-1.52]]


In [62]:
import statsmodels.api as sm
logit_model = sm.Logit(y, x).fit()
logit_model.summary()

Optimization terminated successfully.
         Current function value: 0.192953
         Iterations 10


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                  569
Model:                          Logit   Df Residuals:                      564
Method:                           MLE   Df Model:                            4
Date:                Wed, 29 Jun 2022   Pseudo R-squ.:                  0.7078
Time:                        10:21:16   Log-Likelihood:                -109.79
converged:                       True   LL-Null:                       -375.72
Covariance Type:            nonrobust   LLR p-value:                8.602e-114
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1310      0.427      0.307      0.759      -0.705       0.967
x1             1.7760      3.608      0.492      0.623      -5.296       8.848
x2            -1.1798      0.195     -6.055      0.000      -1.562      -0.798
x3            -7.0555      4.240     -1.664      0.096     -15.366       1.255
x4            -1.5228      0.215     -7.073      0.000      -1.945      -1.101
==============================================================================

Possibly complete quasi-separation: A fraction 0.10 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

In [63]:
model = LogisticRegression()
model.fit(ex[['mean radius', 'mean texture', 'mean area', 'mean symmetry']].values.reshape(-1,4), ex["target"])
print("sklearn library:", model.intercept_, model.coef_)
print("Implementation:",weight[-1].T)

sklearn library: [0.57061904] [[-2.04511424 -1.08127742 -2.07942275 -1.36601857]]
Implementation: [[ 0.13100912  1.77597861 -1.17984702 -7.05548543 -1.52283888]]
